<a href="https://colab.research.google.com/github/riverdogcabin/PSDS4900/blob/main/OWM_pull_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [178]:
import requests
import datetime
import time
import pandas as pd
import json

In [183]:
# historic data API call for OpenWeather
# https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lon}&dt={time}&appid={API key}

In [179]:
with open('/content/drive/MyDrive/Capstone/PSDS4900/config.json') as configuration:
  config = json.load(configuration)
  my_station = config.get('WU')["stationid"]
  my_lat = config.get('OWM')["lat"]
  my_lon = config.get('OWM')["lon"]
  APIKEY = config.get('OWM')["APIkey"]

In [180]:
#get the Weather Underground data into a dataframe, create the hourly timestamp for grouping, etc
df = pd.read_csv('/content/drive/MyDrive/Capstone/PSDS4900/wu_data/wu_observations.csv')
df['timestamp'] = pd.to_datetime(df['epoch'], unit='s')
df = df.assign(timestamp_rounded=df.timestamp.dt.round('60min')) #rounded to the hour
print('with dupes',df.shape)
before = df.shape[0]
df.drop_duplicates(inplace=True)
print('without dupes',df.shape)
print('difference: ',before-df.shape[0])
df = df[df.stationID==my_station] #trim the df down to just my station for purposes of the comparisons
# df.tail()

with dupes (106143, 34)
without dupes (106143, 34)
difference:  0


In [181]:
#get the stations that are most highly correlated with mine (exported from my local DB)
highly_correlated_locations_df = pd.read_csv('/content/drive/MyDrive/Capstone/PSDS4900/strong_correlations.csv')
# highly_correlated_locations_df

In [182]:
#helper def to pull for specific location on specific date
def owm_by_coord_date(lat='39.44805951764288',lon='-104.73659430737496',dt=False):
  url_template = 'https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lon}&dt={time}&appid={APIkey}'
  if(not dt):
    n = datetime.datetime.now()
    dt = str(int(time.mktime(datetime.datetime(n.year, n.month, n.day).timetuple())))
  resp = requests.get(url_template.format(lat=lat,lon=lon,time=dt,APIkey=APIKEY))
  if(resp.status_code != 200):
    return resp.status_code
  resp_dict = eval(resp.text)
  owm_df = pd.DataFrame(resp_dict.get('hourly')).rename(columns={'dt':'epoch'})
  owm_df['timestamp_rounded'] = pd.to_datetime(owm_df['epoch'], unit='s')
  owm_df['lat'] = lat
  owm_df['lon'] = lon
  # print('owm_df.shape',owm_df.shape)
  return owm_df

#helper def to pull a location for a set number of days back, max days back is 5
def get_last_x_days(num_days_back=5,lat='39.44805951764288',lon='-104.73659430737496'):
  return_df = pd.DataFrame()
  today = datetime.date.today()
  start_date = today - datetime.timedelta(days=num_days_back)
  for i in range(num_days_back):
    dt = str(int(time.mktime(start_date.timetuple())))
    print('getting date',start_date)
    # print(dt)
    return_df = return_df.append(owm_by_coord_date(dt=dt,lat=lat,lon=lon))
    # print(return_df.shape)
    time.sleep(2)
    start_date = start_date + datetime.timedelta(days=1)
  return_df.reset_index(inplace=True,drop=True)
  return return_df


In [184]:
#gahter OWM data for each station for max # days back, stashing off in a big dataframe
owm_df = pd.DataFrame()
for i,lat,lon,stationID in highly_correlated_locations_df[['lat','lon','stationID']].itertuples():
  print("Pulling {},{} for station {}".format(lat,lon,stationID))
  temp_df = get_last_x_days(lat=lat,lon=lon)
  temp_df['stationID'] = stationID
  owm_df = owm_df.append(temp_df)

owm_df.shape

Pulling 39.427,-104.79 for station KCOCASTL404
getting date 2021-04-25
getting date 2021-04-26
getting date 2021-04-27
getting date 2021-04-28
getting date 2021-04-29
Pulling 39.430405,-104.705574 for station KCOPARKE148
getting date 2021-04-25
getting date 2021-04-26
getting date 2021-04-27
getting date 2021-04-28
getting date 2021-04-29
Pulling 39.438,-104.717003 for station KCOPARKE320
getting date 2021-04-25
getting date 2021-04-26
getting date 2021-04-27
getting date 2021-04-28
getting date 2021-04-29
Pulling 39.453,-104.715 for station KCOPARKE374
getting date 2021-04-25
getting date 2021-04-26
getting date 2021-04-27
getting date 2021-04-28
getting date 2021-04-29
Pulling 39.448,-104.73700000000001 for station KCOPARKE414
getting date 2021-04-25
getting date 2021-04-26
getting date 2021-04-27
getting date 2021-04-28
getting date 2021-04-29


(600, 18)

In [188]:
#columns that have anything to do with wind measurements
wind_columns = []
for cname in owm_df.columns.to_list()+df.columns.to_list():
  if(cname.startswith('wind')):
    wind_columns.append(cname)

In [189]:
#build big correlation table for comparision
corr_df = pd.DataFrame()
for i,stationID in highly_correlated_locations_df[['stationID']].itertuples():
  # print(stationID)
  big_df = owm_df[owm_df.stationID==stationID].merge(df,on='timestamp_rounded',how='inner')
  # big_df.shape
  wind_df = big_df[wind_columns]
  # wind_df.head()
  temp_df = wind_df.corr().iloc[:3].T.iloc[3:]
  for cname in temp_df.columns:
    corr_df[cname+'_'+stationID] = temp_df[cname]
  # corr_df = corr_df.append(wind_df.corr().iloc[:3].T.iloc[3:])
corr_df

KCOCASTL404
KCOPARKE148
KCOPARKE320
KCOPARKE374
KCOPARKE414


,wind_speed_KCOCASTL404,wind_deg_KCOCASTL404,wind_gust_KCOCASTL404,wind_speed_KCOPARKE148,wind_deg_KCOPARKE148,wind_gust_KCOPARKE148,wind_speed_KCOPARKE320,wind_deg_KCOPARKE320,wind_gust_KCOPARKE320,wind_speed_KCOPARKE374,wind_deg_KCOPARKE374,wind_gust_KCOPARKE374,wind_speed_KCOPARKE414,wind_deg_KCOPARKE414,wind_gust_KCOPARKE414
winddirAvg,0.242257,0.607330,0.308781,0.369389,0.681157,0.390015,0.367960,0.686058,0.396894,0.230563,0.634095,0.285354,0.347759,0.647315,0.419612
windspeedHigh,0.565982,0.789804,0.678799,0.703989,0.794736,0.742069,0.698604,0.802671,0.747289,0.531127,0.735667,0.646204,0.680742,0.830833,0.762642
windspeedLow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
windspeedAvg,0.502207,0.654207,0.615463,0.725455,0.697631,0.765521,0.719256,0.704766,0.769197,0.441168,0.579261,0.574183,0.702168,0.740879,0.780852
windgustHigh,0.539809,0.797046,0.649906,0.718353,0.819043,0.757575,0.712779,0.826337,0.762673,0.498715,0.736146,0.615127,0.694498,0.856075,0.777243
windgustLow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
windgustAvg,0.501899,0.662447,0.605897,0.727654,0.707064,0.758087,0.721337,0.714432,0.761916,0.442356,0.588774,0.564711,0.703545,0.749166,0.773910
windchillHigh,0.303379,0.713619,0.417776,0.465477,0.799293,0.519162,0.465363,0.805579,0.529384,0.276923,0.730194,0.377495,0.446038,0.768869,0.560863
windchillLow,0.311544,0.714088,0.437782,0.465414,0.794275,0.531869,0.466986,0.799970,0.542412,0.279582,0.722857,0.390226,0.450423,0.765092,0.574501
windchillAvg,0.312740,0.721432,0.436061,0.465632,0.801130,0.527730,0.466253,0.807142,0.538091,0.285150,0.734560,0.392646,0.448171,0.772029,0.569693


In [190]:
#OWM measurements age off, just stashing off all the data so I can reference it later on if necessary
owm_df.to_csv('/content/drive/MyDrive/Capstone/PSDS4900/owm_data/20210425_20210429_highly_correlated.csv')